<a href="https://colab.research.google.com/github/AnkitaChakraborty1/Capstone_Adversarial_Robustness/blob/main/Albert_Goal_TransformationList_GreedyWord_Custom_Recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##pip installs

In [2]:
pip install transformers

     |████████████████████████████████| 3.8 MB 3.2 MB/s 
     |████████████████████████████████| 6.5 MB 49.7 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 895 kB 45.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip3 install textattack[tensorflow]

     |████████████████████████████████| 373 kB 3.2 MB/s 
     |████████████████████████████████| 69 kB 7.3 MB/s 
     |████████████████████████████████| 60 kB 7.3 MB/s 
     |████████████████████████████████| 312 kB 54.0 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 769 kB 55.8 MB/s 
     |████████████████████████████████| 322 kB 42.4 MB/s 
     |████████████████████████████████| 454.3 MB 17 kB/s 
     |████████████████████████████████| 4.9 MB 46.0 MB/s 
     |████████████████████████████████| 462 kB 50.9 MB/s 
     |████████████████████████████████| 125 kB 61.5 MB/s 
     |████████████████████████████████| 4.0 MB 45.2 MB/s 
     |████████████████████████████████| 132 kB 56.9 MB/s 
     |████████████████████████████████| 14.8 MB 42.0 MB/s 
     |████████████████████████████████| 1.2 MB 53.1 MB/s 
     |████████████████████████████████| 4.9 MB 50.4 MB/s 
     |████████████████████████████████| 4.9 MB 48.3 MB/s 
     |████████████

In [2]:
pip install --upgrade numpy

In [1]:
pip install folium==0.2.1

## Imports

In [3]:
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from textattack import Attack
from textattack.constraints.pre_transformation import (
        InputColumnModification,
        MaxModificationRate,
        RepeatModification,
        StopwordModification,
        MaxWordIndexModification
                )
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification, TargetedClassification, InputReduction
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import (
   
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapWordNet,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapRandomCharacterSubstitution,
    WordSwapHomoglyphSwap,
    WordSwapEmbedding,
    WordSwapHowNet,
    WordSwapInflections,
    WordDeletion,
    WordSwapGradientBased,
    WordSwapExtend,
    WordSwapContract,
    WordSwapChangeName,
    WordSwapChangeNumber,
    WordSwapChangeLocation,
    WordSwapRandomCharacterSubstitution,
    WordSwapMaskedLM,
    WordInsertionMaskedLM,
    WordMergeMaskedLM,
)
from textattack.constraints.overlap import LevenshteinEditDistance
#from textattack.attack_recipe import AttackRecipe

from abc import ABC, abstractmethod
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from IPython.core.display import display, HTML

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:14<00:00, 33.5MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp8wrr6h4d.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [4]:
from textattack.datasets import HuggingFaceDataset
from textattack.attacker import Attacker
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.goal_functions import TargetedClassification
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper
import torch

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/albert-base-v2-imdb")
# We wrap the model so it can be used by textattack
model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)
#dataset = HuggingFaceDataset("imdb", split="test")
#dataset = HuggingFaceDataset("glue", "sst2", "train")

Using custom data configuration default


textattack: Loading datasets dataset rotten_tomatoes, split test.


Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a. Subsequent calls will reuse this data.


##Custom Recipes

In [5]:
class Recipe(Attack,ABC):
    @staticmethod
    def build(model_wrapper,transf,gf):
        if(transf == "WordSwapWordNet"):
            transformation = WordSwapWordNet()
        elif(transf == "WordSwapQWERTY"):
            transformation = WordSwapQWERTY()
        elif(transf == "WordSwapHomoglyphSwap"):
            transformation = WordSwapHomoglyphSwap()
        elif(transf == "WordSwapNeighboringCharacterSwap"):
            transformation = WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        elif(transf == "WordSwapRandomCharacterInsertion"):
            transformation = WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    )
        elif(transf == "WordSwapRandomCharacterDeletion"):
             transformation = WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    )
        elif(transf == "WordSwapEmbedding"):
             transformation = WordSwapEmbedding(max_candidates=5)
        elif(transf == "WordSwapHowNet"):
             transformation = WordSwapHowNet()
        elif(transf == "WordSwapInflections"):
             transformation = WordSwapInflections()
        elif(transf == "WordDeletion"):
             transformation = WordDeletion()
        elif(transf == "WordSwapGradientBased"):
             transformation = WordSwapGradientBased(model_wrapper, top_n=1)
        elif(transf == "WordSwapExtend"):
             transformation = WordSwapExtend()
        elif(transf == "WordSwapContract"):
             transformation = WordSwapContract()
        elif(transf == "WordSwapChangeName"):
             transformation = WordSwapChangeName()
        elif(transf == "WordSwapChangeNumber"):
             transformation = WordSwapChangeNumber()  
        elif(transf == "WordSwapChangeLocation"):
             transformation = WordSwapChangeLocation()
        elif(transf == "WordSwapRandomCharacterSubstitution"):
             transformation = WordSwapRandomCharacterSubstitution()
        elif(transf == "WordSwapMaskedLM"):
             transformation = WordSwapMaskedLM(method="bert-attack", max_candidates=48)
        elif(transf == "WordInsertionMaskedLM"):
             transformation = WordInsertionMaskedLM(
                    #masked_language_model=shared_masked_lm,
                    #tokenizer=shared_tokenizer,
                    max_candidates=50,
                    min_confidence=0.0,
                )
        elif(transf == "WordMergeMaskedLM"):
             transformation = WordMergeMaskedLM(
                    #masked_language_model=shared_masked_lm,
                    #tokenizer=shared_tokenizer,
                    max_candidates=50,
                    min_confidence=5e-3,
                )

        constraints = [RepeatModification(), StopwordModification()]
        if(gf == "TC"):
            goal_function = TargetedClassification(model_wrapper)
        elif(gf == "IR"):
            goal_function = InputReduction(model_wrapper)
        elif(gf == "UTC"):
            goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR("weighted-saliency")
        return Attack(goal_function, constraints, transformation, search_method)


InputReduction - 'WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
'WordSwapRandomCharacterDeletion','WordSwapEmbedding',
'WordSwapHowNet','WordDeletion','WordMergeMaskedLM',
               'WordSwapRandomCharacterSubstitution'

In [7]:
transf_list = ['WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
               'WordSwapRandomCharacterDeletion','WordSwapEmbedding',
               'WordSwapHowNet','WordDeletion','WordMergeMaskedLM',
               'WordSwapRandomCharacterSubstitution','WordSwapMaskedLM',
               'WordInsertionMaskedLM','WordMergeMaskedLM']
gf_list = ['IR','TC','UTC']
              
for gf in gf_list:         
    for transf in transf_list:
        print("Starting attack for-----------------",transf,gf)
        try:
            attack = Recipe.build(model_wrapper,transf,gf)
            attack_args = AttackArgs(num_examples=8,)
            attacker = Attacker(attack, dataset,attack_args)
            attack_results = attacker.attack_dataset()
        except ValueError:
            print("Not compatible")
            pass

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.
  0%|          | 0/8 [00:00<?, ?it/s]

Starting attack for----------------- WordSwapWordNet IR
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  InputReduction(
    (maximizable):  False
    (target_num_words):  1
  )
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  12%|█▎        | 1/8 [00:07<00:50,  7.26s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .




[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:  38%|███▊      | 3/8 [00:07<00:13,  2.62s/it]

...


Starting attack for----------------- WordInsertionMaskedLM UTC


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Not compatible
Starting attack for----------------- WordMergeMaskedLM UTC


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Not compatible


In [6]:
transf_list = ['WordInsertionMaskedLM','WordMergeMaskedLM','WordSwapExtend','WordSwapContract',
              'WordSwapChangeName','WordSwapChangeNumber','WordSwapChangeLocation']
gf_list = ['IR','TC','UTC']
              
for gf in gf_list:         
    for transf in transf_list:
        print("Starting attack for-----------------",transf,gf)
        try:
            attack = Recipe.build(model_wrapper,transf,gf)
            attack_args = AttackArgs(num_examples=8,)
            attacker = Attacker(attack, dataset,attack_args)
            attack_results = attacker.attack_dataset()
        except ValueError:
            print("Not compatible")
            pass

Starting attack for----------------- WordInsertionMaskedLM IR


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.


Not compatible
Starting attack for----------------- WordMergeMaskedLM IR


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatib

...

--------------------------------------------- Result 4 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .


--------------------------------------------- Result 5 ---------------------------------------------
[[Positive (74%)]] --> [[[FAILED]]]

red dragon " never cuts corners .


--------------------------------------------- Result 6 ---------------------------------------------
[[Negative (64%)]] --> [[[SKIPPED]]]

fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .




[Succeeded / Failed / Skipped / Total] 0 / 5 / 2 / 7:  88%|████████▊ | 7/8 [00:01<00:00,  4.17it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

throws in enough clever and unexpected twists to make the formula feel fresh .




[Succeeded / Failed / Skipped / Total] 0 / 6 / 2 / 8: 100%|██████████| 8/8 [00:02<00:00,  3.95it/s]

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (97%)]] --> [[[FAILED]]]

weighty and ponderous but every bit as filling as the treat of the title .



+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 6     |
| Number of skipped attacks:    | 2     |
| Original accuracy:            | 75.0% |
| Accuracy under attack:        | 75.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 15.0  |
| Avg num queries:              | 13.33 |
+-------------------------------+-------+


In [8]:
transf_list = ['WordSwapWordNet','WordSwapHomoglyphSwap','WordSwapNeighboringCharacterSwap',
               'WordSwapRandomCharacterDeletion','WordSwapEmbedding',
               'WordSwapHowNet','WordDeletion','WordMergeMaskedLM',
               'WordSwapRandomCharacterSubstitution','WordSwapMaskedLM',
               'WordInsertionMaskedLM','WordMergeMaskedLM']
gf_list = ['IR','TC','UTC']
              
for gf in gf_list:         
    for transf in transf_list:
        print("Starting attack for-----------------",transf,gf)
        try:
            attack = Recipe.build(model_wrapper,transf,gf)
            attack_args = AttackArgs(num_examples=100,)
            attacker = Attacker(attack, dataset,attack_args)
            attack_results = attacker.attack_dataset()
        except ValueError:
            print("Not compatible")
            pass
        except KeyError:
            print("Key Error")
            pass

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.

  0%|          | 0/100 [00:00<?, ?it/s]

Starting attack for----------------- WordSwapWordNet IR
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  InputReduction(
    (maximizable):  False
    (target_num_words):  1
  )
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 




  1%|          | 1/100 [00:04<07:23,  4.48s/it]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   1%|          | 1/100 [00:04<07:24,  4.49s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .





[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   2%|▏         | 2/100 [00:04<03:57,  2.42s/it]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   2%|▏         | 2/100 [00:04<03:57,  2.42s/it]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 1 / 3:   3%|▎         | 3/100 [00:04<02:37,  1.63s/it]

...


[Succeeded / Failed / Skipped / Total] 0 / 90 / 10 / 100: 100%|██████████| 100/100 [03:55<00:00,  2.36s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[Positive (96%)]] --> [[[FAILED]]]

an unusually dry-eyed , even analytical approach to material that is generally played for maximum moisture .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 90     |
| Number of skipped attacks:    | 10     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 90.0%  |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 18.45  |
| Avg num queries:              | 114.67 |
+-------------------------------+--------+
Starting attack for----------------- WordSwapHowNet IR



textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.

  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  InputReduction(
    (maximizable):  False
    (target_num_words):  1
  )
  (transformation):  WordSwapHowNet(
    (max_candidates):  -1
  )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



KeyError: ignored